# Author: Seunghee Kim
- Created on: 2024-11-20

## Notebook Explain  
1_dataset_preprocess.ipynb의 결과로 나오게 된 df_human_train.csv, df_human_valid.csv, df_human_test.csv 데이터셋에 대해서, 동일한 'essay_prompt'를 OpenAI의 GPT-4o-mini 모델에게 제공해서 AI-Generated Text를 얻는 코드

# Import Library

In [ ]:
import os
import pandas as pd
from tqdm import tqdm
import yaml
from openai import OpenAI

# Config & Path

In [ ]:
class CFG:
    SEED = 1
    MODEL = "gpt-4o-mini"
    
    # api key가 담긴 yaml파일 (해당 yaml 파일은 .gitignore에 반드시 추가해야 함!!!)
    API_CONFIG_PATH = './config.yaml'
    
    # 1_dataset_preprocess.ipynb의 결과로 나온 전처리된 Human Dataset 경로
    DF_HUMAN_TRAIN_PATH = './df_human_train.csv'
    DF_HUMAN_VALID_PATH = './df_human_valid.csv'
    DF_HUMAN_TEST_PATH = './df_human_test.csv'
    
    # AI-Generated Text까지 포함된 csv파일의 output 경로
    DF_HUMAN_AI_TRAIN_PATH = './df_human_ai_train.csv'
    DF_HUMAN_AI_VALID_PATH = './df_human_ai_valid.csv'
    DF_HUMAN_AI_TEST_PATH = './df_human_ai_test.csv'
    
    # Final Dataset output 경로 (Human Text, AI Text, Label 존재)
    DF_FINAL_TRAIN_PATH = './df_final_train.csv'
    DF_FINAL_VALID_PATH = './df_final_valid.csv'
    DF_FINAL_TEST_PATH = './df_final_test.csv'

# Load OpenAI API Key

In [ ]:
# yaml파일에 존재하는 api key를 load 하는 함수
def load_api_key(yaml_file):
    with open(yaml_file, 'r', encoding='utf-8') as file:
        config = yaml.safe_load(file)
    return config['openai']['api_key_ksh']

api_key = load_api_key(CFG.API_CONFIG_PATH)
client = OpenAI(api_key=api_key)

# Make AI-Generated Text

### Train Dataset

In [ ]:
train_dataset = pd.read_csv(CFG.DF_HUMAN_TRAIN_PATH)

# test
train_dataset = train_dataset.head(3)

train_dataset['AI_Generated_Text'] = ""

for idx, row in tqdm(train_dataset.iterrows(), total=len(train_dataset)):
    input_text = row['essay_prompt']
    student_grade = row['student_grade']
    school, grade = student_grade.split('_')
    if school == '중등':
        school = school[0] # 중등인 경우 '중'만 가져옴
    
    # 학교 수준에 따라 다르게 설정하는 시스템 프롬프트
    # ex) "너는 대한민국의 고등학교 2학년 학생이다."
    grade_system_prompt = f"너는 대한민국의 {school}학교 {grade} 학생이다."
    # print(grade_system_prompt)
    # print('=================')
    # print(input_text)
    # print('=================')
    response = client.chat.completions.create(
        model=CFG.MODEL,
        messages=[
            {"role": "system", "content": grade_system_prompt},
            {"role": "user", "content": input_text}
        ]
    )
    
    llm_output = response.choices[0].message.content
    # print(llm_output)
    # print('=================')
    # LLM이 생성한 Text를 'AI_Generated_Text' 열에 추가
    train_dataset.at[idx, 'AI_Generated_Text'] = llm_output

train_dataset.to_csv(CFG.DF_HUMAN_AI_TRAIN_PATH, index=False)

### Valid Dataset

In [ ]:
valid_dataset = pd.read_csv(CFG.DF_HUMAN_VALID_PATH)
valid_dataset['AI_Generated_Text'] = ""


for idx, row in tqdm(valid_dataset.iterrows(), total=len(valid_dataset)):
    input_text = row['essay_prompt']
    student_grade = row['student_grade']
    school, grade = student_grade.split('_')
    if school == '중등':
        school = school[0] # 중등인 경우 '중'만 가져옴
    
    # 학교 수준에 따라 다르게 설정하는 시스템 프롬프트
    # ex) "너는 대한민국의 고등학교 2학년 학생이다."
    grade_system_prompt = f"너는 대한민국의 {school}학교 {grade} 학생이다."
    # print(grade_system_prompt)
    # print('=================')
    # print(input_text)
    # print('=================')
    response = client.chat.completions.create(
        model=CFG.MODEL,
        messages=[
            {"role": "system", "content": grade_system_prompt},
            {"role": "user", "content": input_text}
        ]
    )
    
    llm_output = response.choices[0].message.content
    # print(llm_output)
    # print('=================')
    # LLM이 생성한 Text를 'AI_Generated_Text' 열에 추가
    valid_dataset.at[idx, 'AI_Generated_Text'] = llm_output


valid_dataset.to_csv(CFG.DF_HUMAN_AI_VALID_PATH, index=False)

### Test Dataset

In [ ]:
test_dataset = pd.read_csv(CFG.DF_HUMAN_TEST_PATH)
test_dataset['AI_Generated_Text'] = ""


for idx, row in tqdm(test_dataset.iterrows(), total=len(test_dataset)):
    input_text = row['essay_prompt']
    student_grade = row['student_grade']
    school, grade = student_grade.split('_')
    if school == '중등':
        school = school[0] # 중등인 경우 '중'만 가져옴
    
    # 학교 수준에 따라 다르게 설정하는 시스템 프롬프트
    # ex) "너는 대한민국의 고등학교 2학년 학생이다."
    grade_system_prompt = f"너는 대한민국의 {school}학교 {grade} 학생이다."
    # print(grade_system_prompt)
    # print('=================')
    # print(input_text)
    # print('=================')
    response = client.chat.completions.create(
        model=CFG.MODEL,
        messages=[
            {"role": "system", "content": grade_system_prompt},
            {"role": "user", "content": input_text}
        ]
    )
    
    llm_output = response.choices[0].message.content
    # print(llm_output)
    # print('=================')
    # LLM이 생성한 Text를 'AI_Generated_Text' 열에 추가
    test_dataset.at[idx, 'AI_Generated_Text'] = llm_output


test_dataset.to_csv(CFG.DF_HUMAN_AI_TEST_PATH, index=False)

# Build Final Custom Dataset